In [8]:
using Rotations, LinearAlgebra
include("../src/helper_funcs.jl")

inv_fourier (generic function with 1 method)

# Modeling globular protein monomers and their interactions

The goal is to create a pairwise interaction between two actin monomers, that not only depend on their distance, but also their relative orientations (the angle between their directrices?).

This requires a special data structure for each monomer, that contains its coordinates in 3D but also 3 vectors that "orient" the monomer.

In [9]:
mutable struct Monomer
    coord::Vector{Float64}
    orientation::Matrix{Float64} #3 unit vectors
    radius::Float64 #hard sphere radius
    patch_orientations::Matrix{Float64} #orientations of the attractive patches on the surface of the hard sphere, unit vectors
    patch_radii::Vector{Float64} #radii of the attractive patches
end

Now the "distance" between two monomers is not just their Euclidean distance, but also the difference between their orientations, characterized by the rotation matrix $R$ that transforms one orientation to another.
$RA = B$, where $R$ is the rotation matrix that we want to solve. We have $R = BA^{-1}$.

So the pair potential is basically $v_{ij}(\Delta \mathbf{r}_{ij}, R_{ij})$. This is way too complicated as it depends on 12 scalars (3 for the vector, 9 for the rotation matrix). To simplify the interactions, we use the patchy particle approximation.

In [ ]:
function pair_energy(m1::Monomer, m2::Monomer, l::Float64, patch_energy_func::Function)
    println(dist_pbc(m1.coord, m2.coord, l))
    if dist_pbc(m1.coord, m2.coord, l) < (m1.radius + m2.radius)  #the hard core interaction
        return Inf
    end
    energy = 0.0
    patch_positions1 = m1.orientation * m1.patch_orientations
    patch_positions2 = m2.orientation * m2.patch_orientations
    for i in axes(patch_positions1, 2), j in axes(patch_positions2, 2)
        real_patchi = m1.coord + patch_positions1[:, i]
        real_patchj = m2.coord + patch_positions2[:, j]
        patch_distance = dist_pbc(real_patchi, real_patchj, l)
        energy += patch_energy_func(patch_distance, m1.patch_radii[i] + m2.patch_radii[j])
    end
    return energy
end

function my_patch_energy(r, R) # Example patch energy function: Gaussian attraction
    return -exp(-(r/R)^2) 
end

function rand_unit_vector()
    v = randn(3)
    return v / norm(v)
end

In [ ]:
# create a monomer m1 with orientation given by the identity matrix, and with 3 patches whose orientations are random and whose radii are 0.1   

m1 = Monomer(
    [0.0, 0.0, 0.0], 
    Matrix(Float64.(I(3))), 
    0.5, 
    hcat(rand_unit_vector(), rand_unit_vector(), rand_unit_vector()), 
    [0.2, 0.2, 0.2]
)

Monomer([0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], 0.5, [-0.8242047931952842 0.3496893501350819 0.7934446494526936; 0.46922726792482355 -0.9360848160715747 -0.5445339724123435; 0.3170366381188848 0.03824363374929397 -0.27189766667576465], [0.2, 0.2, 0.2])

In [29]:
m2 = deepcopy(m1)

Monomer([0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], 0.5, [-0.8242047931952842 0.3496893501350819 0.7934446494526936; 0.46922726792482355 -0.9360848160715747 -0.5445339724123435; 0.3170366381188848 0.03824363374929397 -0.27189766667576465], [0.2, 0.2, 0.2])

In [52]:
m2.coord = [1, 0., 0.0] # Move m2 to a different position
m2.orientation = rand(RotMatrix{3,Float64})*m2.orientation # Randomly rotate m2

3×3 Matrix{Float64}:
  0.480634   -0.575903  0.661306
  0.0244254   0.762622  0.646383
 -0.876581   -0.294521  0.380609

In [53]:
pair_energy(m1, m2, 10.0, my_patch_energy)

1.0


-0.01897119685732814

In [ ]:
#generate a small random rotation matrix for Monte Carlo rotation moves
function small_random_rotation(max_angle::Float64)
    axis = randn(3)
    axis /= norm(axis)                # random unit vector
    angle = rand() * max_angle        # random angle in [0, max_angle]
    return RotMatrix(AngleAxis(angle, axis[1], axis[2], axis[3]))
end

# Example usage:
R = small_random_rotation(0.05)

3×3 RotMatrix3{Float64} with indices SOneTo(3)×SOneTo(3):
  0.999466    0.0177427  -0.0274211
 -0.0174599   0.999792    0.0105161
  0.027602   -0.0100317   0.999569

In [10]:
# monte carlo simulation of a system of monomers that implements both translation and rotation moves
# use keyword arguments for clarity
function monte_carlo_simulation(monomers::Vector{Monomer}, l::Float64; 
                                 num_steps::Int = 1000, 
                                 max_translation::Float64 = 0.1, 
                                 max_rotation::Float64 = 0.05, 
                                 patch_energy_func::Function = my_patch_energy)
    for step in 1:num_steps
        # Randomly select a monomer to move
        idx = rand(1:length(monomers))
        m = monomers[idx]

        # Random translation move
        translation = randn(3) * max_translation
        new_coord = m.coord + translation
        new_coord = pbc(new_coord, l)  # Apply periodic boundary conditions

        # Random rotation move
        rotation = small_random_rotation(max_rotation)
        new_orientation = rotation * m.orientation

        # Create a new monomer with the proposed changes
        new_monomer = Monomer(new_coord, new_orientation, m.radius, m.patch_orientations, m.patch_radii)

        # Calculate the energy of the new configuration
        energy_new = 0.0
        for other_m in monomers
            if other_m != m
                energy_new += pair_energy(new_monomer, other_m, l, patch_energy_func)
            end
        end

        # Calculate the energy of the old configuration
        energy_old = 0.0
        for other_m in monomers
            if other_m != m
                energy_old += pair_energy(m, other_m, l, patch_energy_func)
            end
        end

        # Metropolis criterion for accepting the move
        if energy_new < energy_old || rand() < exp(-(energy_new - energy_old))
            monomers[idx] = new_monomer  # Accept the move
        end
    end

    return monomers
end     

monte_carlo_simulation (generic function with 1 method)